# Python Data, 2024

---

* [Spojování dat](#Spojov%C3%A1n%C3%AD-dat),
    - [spojování s CONCAT](#Spojování-s-CONCAT),
    - [spojování s CONCAT a JOIN](#concat-a-volitelný-argument-join),
    - [spojování s metodou APPEND](#Spojování-pomocí-metody-APPEND),
    - [spojování s MERGE](#Spojování-pomocí-merge),
    - [spojování s metodou JOIN](#Spojení-pomocí-indexů,-join),
* [Agregace](#Agregace),
    - [jednoduchá agregace](#Jednoduchá-agregace),
    - [seskupování GROUPBY](#Seskupování-groupby),
    - [agregace](#Agregace-s-metodami),
    - [filtrování](#Filtrování),
    - [transformace](#Transformace),
    - [apply](#Metoda-apply),
* [Pivot tabulka](#Pivot-tabulky),
    - [úvodní motivace](),
    - [syntaxe tabulky](),
    - [doplňující možnosti]().
---



<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.APOjuHGvkM0wQaUx9ELKAgHaHa%26pid%3DApi&f=1&ipt=a8b2b692c40e190d4b28c164bf13e6c944702a69944035a4ef9053aa6ea1b190&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

<br>

## Spojování dat

---

Takovými operacemi si můžeš představit jednoduché operace jako **spojování** (*konkatenace*) **dvou a více datasetů** až po složitější *JOINY* podobně jako u databází.

`pandas` obsahuje různé *funkce* a *metody*, které umožňují tento způsob práce.

* funkce `concat`,
* funkce `merge`,
* metoda `append` (*outdated*),
* metoda `join`.

### Spojování s CONCAT

---

Funkce `concat()` v `pandas` umožňuje spojit **dva nebo více**:
1. **sloupečků** tabulky,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,
3. **tabulek** samotných,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,

Použití funkce `concat()` spočívá **v předání seznamu objektů**, které chceš spojit, a parametru `axis`, který určuje osu, podle které se májí objekty spojit:

#### Spojení sloupečků, CONCAT

---

In [1]:
from pandas import concat, Series, DataFrame

In [2]:
sloupec_1 = Series(['A', 'B', 'C'], index=[1, 2, 3])
sloupec_2 = Series(['D', 'E', 'F'], index=[4, 5, 6])

In [3]:
sloupec_1

1    A
2    B
3    C
dtype: object

In [4]:
sloupec_2

4    D
5    E
6    F
dtype: object

In [6]:
# spojene_sloupce = concat(sloupec_1, sloupec_2, axis=0)  # --> "TypeError"

<br>

Spojení **vertikální** (pod sebou):

In [8]:
spojene_sloupce_vertikalne = concat((sloupec_1, sloupec_2), axis=0)

In [10]:
# concat?

In [12]:
spojene_sloupce_vertikalne

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

<br>

Spojení **horizontální** (vedle sebe):

In [11]:
horizontalni_spoj = concat([sloupec_1, sloupec_2], axis=1)

In [13]:
horizontalni_spoj

,0,1
1,A,NaN
2,B,NaN
3,C,NaN
4,NaN,D
5,NaN,E
6,NaN,F


<br>

#### Spojení tabulek, CONCAT

---

In [14]:
uzivatele_tab_1 = {'jmeno': ['Matouš', 'Marek', 'Lukáš'],
               'vek': [25, 30, 35]}

In [15]:
uzivatele_tab_2 = {'jmeno': ['Petr', 'Jan', 'Michal'],
                   'vek': [40, 45, 50]}

In [16]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [17]:
uzivatele_tab_1_df

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35


In [18]:
uzivatele_tab_2_df

,jmeno,vek
0,Petr,40
1,Jan,45
2,Michal,50



#### Samotné spojení dvou tabulek

---

Stejně jako u sloupečků, nezapomínat na poziční argumenty (závorka).

Stejně tak, vybrat správnou osu.

In [21]:
spojene_tab_df = concat([uzivatele_tab_1_df, uzivatele_tab_2_df], axis=0)

In [22]:
spojene_tab_df

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35
0,Petr,40
1,Jan,45
2,Michal,50


#### Co s duplicitními Indexy?

---

Tady se ovšem **zduplikovali hodnoty** některých indexů.

Ty je potřeba opravit tímto postupem:
1. **Vytvořím nový sloupeček** pro Indexy,
2. **odstraním starý sloupeček** s duplicitami.

In [23]:
spojene_tab_df = spojene_tab_df.reset_index()  # Přidá nový Index

In [24]:
spojene_tab_df

,index,jmeno,vek
0,0,Matouš,25
1,1,Marek,30
2,2,Lukáš,35
3,0,Petr,40
4,1,Jan,45
5,2,Michal,50


In [25]:
spojene_tab_df = spojene_tab_df.drop("index", axis=1)

In [26]:
spojene_tab_df

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35
3,Petr,40
4,Jan,45
5,Michal,50


<br>

Kontrola nového rozsahu **pro Index**:

In [27]:
spojene_tab_df.index

RangeIndex(start=0, stop=6, step=1)

<br>

Pozůstatky sloupečků:

In [28]:
spojene_tab_df.columns

Index(['jmeno', 'vek'], dtype='object')

<br>

Nastavení nových Indexů **podle jmen**:

In [29]:
# spojene_tab_df.index.names?

<br>

Doplníš jméno **pro Index**:

In [30]:
spojene_tab_df.index.names = ["Index ze jmén"]

In [31]:
spojene_tab_df.index.names

FrozenList(['Index ze jmén'])

In [32]:
spojene_tab_df.index

RangeIndex(start=0, stop=6, step=1, name='Index ze jmén')

In [33]:
spojene_tab_df

,jmeno,vek
Index ze jmén,,
0,Matouš,25
1,Marek,30
2,Lukáš,35
3,Petr,40
4,Jan,45
5,Michal,50


Pro odchytávání **duplicitních indexů** můžeš doplnit parametr `verify_integrity=True`, případně pokud je irelevantní, ignorovat jej úplně `ignore_index=True`.

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí CONCAT**

In [56]:
from typing import Dict

from pandas import DataFrame


data_populace: Dict[str, list] = {
    "City": ["New York", "London", "Tokyo"],
    "Population": [8.4, 9.0, 13.9],
}

data_plocha = {
    "City": ["New York", "London", "Tokyo"],
    "Area": [783, 1572, 2194],
}

data_kontinent = {
    "City": ["New York", "London", "Tokyo"],
    "Continent": ["North America", "Europe", "Asia"],
}

# Vytvoř funkci "spoj_data_mest", která spojí 3 předchozí DF

In [57]:
populace_df = DataFrame(data_populace)
plocha_df = DataFrame(data_plocha)
kontinent_df = DataFrame(data_kontinent)


def spoj_data_mest(populace: DataFrame,
                   plocha: DataFrame,
                   kontinent: DataFrame) -> DataFrame:
    return concat([populace.set_index('City'),
                   plocha.set_index('City'),
                   kontinent.set_index('City')], axis=1)

In [58]:
spojeny_vystup = spoj_data_mest(
    populace=populace_df,
    plocha=plocha_df,
    kontinent=kontinent_df
)

In [59]:
spojeny_vystup

,Population,Area,Continent
City,,,
New York,8.4,783,North America
London,9.0,1572,Europe
Tokyo,13.9,2194,Asia


<details>
    <summary>▶️ Řešení</summary>
    
```python
from pandas import DataFrame, concat

data_populace = {
    "City": ["New York", "London", "Tokyo"],
    "Population": [8.4, 9.0, 13.9],
}

data_plocha = {
    "City": ["New York", "London", "Tokyo"],
    "Area": [783, 1572, 2194],
}

data_kontinent = {
    "City": ["New York", "London", "Tokyo"],
    "Continent": ["North America", "Europe", "Asia"],
}

populace_df = DataFrame(data_populace)
plocha_df = DataFrame(data_plocha)
kontinent_df = DataFrame(data_kontinent)


def spoj_data_mest(populace, plocha, kontinent):
    """
    Spojí tři tabulky horizontálně podle sloupce City.
    """
    return concat([populace.set_index("City"),
                      plocha.set_index("City"),
                      kontinent.set_index("City")], axis=1).reset_index()


result = spoj_data_mest(populace_df, plocha_df, kontinent_df)
print(result)
```
</details>

<img src="https://imgs.search.brave.com/wR9sz6BPwR_LCDm6Esd_B9VgUBSbW4wbTPi2T9DrdBc/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQwLzM1LzIx/LzM2MF9GXzg0MDM1/MjE1Ml84N1N2T0kx/REM2RElCaHkxc0xz/VnpnYmNIUk1UZno0/cC5qcGc" width="230" style="margin-left:auto; margin-right:auto"/>

### CONCAT a volitelný argument JOIN

---

V jednoduchých ukázkách, jako jsou ty výše, stačilo **tabulky a sloupečky spojit**.

To prakticky není vždy ideální řešení, protože některé sloupečky můžou, **ale nemusí být shodné**.

In [60]:
from pandas import DataFrame, concat

In [61]:
data_tab_1_df = DataFrame(
    {"A": ["A1", "A2"], "B": ["B1", "B2"], "C": ["C1", "C2"]},
    index=[1, 2]
)

In [62]:
data_tab_2_df = DataFrame(
    {"B": ["B3", "B4"], "C": ["C3", "C4"], "D": ["D3", "D4"]},
    index=[3, 4]
)

In [63]:
data_tab_1_df

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [64]:
data_tab_2_df

,B,C,D
3,B3,C3,D3
4,B4,C4,D4



#### Horizontální spojení dvou tabulek

---

In [65]:
vystup_tab_df = concat([data_tab_1_df, data_tab_2_df])  # axis=0

In [66]:
vystup_tab_df

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


Pokud některá data chybějí, jsou automaticky vyplněná **neznámými hodnotami**.

Řešením takové situace můžeš být zavedení jiného **způsobu spojování**.

Tedy nepoužívat defaultní argument `join='outer'`, ale `join='inner'`:

In [67]:
vystup_bez_nan_df = concat([data_tab_1_df, data_tab_2_df],
                           join='inner',
                           axis=0)

In [68]:
vystup_bez_nan_df

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


In [73]:
vystup_vc_nan_df = concat([data_tab_1_df, data_tab_2_df],
                           join='outer',
                           axis=0)

In [74]:
vystup_vc_nan_df

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


<img src="https://imgs.search.brave.com/IZ8jkD1GaDV2tXwgwzM1Xawk-jBMejygby1KcermQp8/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQ2LzQ3LzE1/LzM2MF9GXzg0NjQ3/MTU2MF95Y281RnhM/UG5RR1c4NW1OWHBD/UmcxUWptSzRiMnp3/eS5qcGc" width="200" style="margin-left:auto; margin-right:auto"/>



### Spojování pomocí `MERGE`

---

Další funkcí pro spojování `DataFrame` objektů je `merge`.

Tato funkce je vhodná pro spojování DataFrame objektů, které **mají společné sloupce**.

Můžeš lépe zadávat typ spojení (parametr `how='inner' | 'outer' | 'left' | 'join'`).

Dále ti umožní **definovat sloupec, nebo sloupce**, na kterých chceš spojení provést (parametr `on`).

In [75]:
from pandas import merge

In [76]:
uzivatele_tab_1 = {
    'jmeno': ['Alice', 'Bob', 'Charlie', 'David'],
    'vek': [25, 30, 35, 40],
    'mesto': ['Brno', 'Praha', 'Plzen', 'Ostrava']
}

In [77]:
uzivatele_tab_2 = {
    'jmeno': ['Alice', 'David', 'Emma', 'Frank'],
    'pocet_prijemcu': [100, 200, 150, 250]
}

In [78]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [79]:
uzivatele_tab_1_df

,jmeno,vek,mesto
0,Alice,25,Brno
1,Bob,30,Praha
2,Charlie,35,Plzen
3,David,40,Ostrava


In [80]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


#### Spojení FULL JOIN

---

In [81]:
mergnute_df = merge(uzivatele_tab_1_df,
                    uzivatele_tab_2_df,
                    on='jmeno',
                    how='outer')

In [82]:
mergnute_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25.0,Brno,100.0
1,Bob,30.0,Praha,NaN
2,Charlie,35.0,Plzen,NaN
3,David,40.0,Ostrava,200.0
4,Emma,NaN,NaN,150.0
5,Frank,NaN,NaN,250.0


#### Spojené INNER MERGE

---

In [85]:
mergnute_inner_df = merge(uzivatele_tab_1_df,
                          uzivatele_tab_2_df,
                          on='jmeno',
                          how='inner')

In [86]:
mergnute_inner_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25,Brno,100
1,David,40,Ostrava,200


Funkce sama **doplní přípony**, aby rozlišila **mezi oběma původními sloupci**.

**Pokud potřebuješ vlastní přípony**, můžeš vyzkoušet volitelný argument pro `suffixes`:

In [87]:
df8 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [1, 2, 3, 4]
})

In [88]:
df9 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [3, 1, 4, 2]
})

In [89]:
df8

,jmeno,poradi
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [90]:
df9

,jmeno,poradi
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [91]:
vystup_konflikt = merge(df8, df9, on="jmeno")

In [92]:
vystup_konflikt

,jmeno,poradi_x,poradi_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [93]:
vystup_vlastni_pripony = merge(df8, df9, on='jmeno', suffixes=('_DF8', '_DF9'))

In [94]:
vystup_vlastni_pripony

,jmeno,poradi_DF8,poradi_DF9
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Obecně platí, že pokud potřebuješ *spojit* dva nebo více `DataFrame` objektů podle **společného sloupce nebo sloupců**, použij funkci `merge()`.

Pokud chceš jenom **přidat další řádky nebo sloupce** do existujícího `DataFrame` objektu, použij funkci `concat`.

#### Ukázka MERGE funkce s dopomocí `LEFT`

In [95]:
uzivatele_tab_1_df

,jmeno,vek,mesto
0,Alice,25,Brno
1,Bob,30,Praha
2,Charlie,35,Plzen
3,David,40,Ostrava


In [96]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


In [97]:
vystup_left_join_df = merge(uzivatele_tab_1_df,
                            uzivatele_tab_2_df,
                            on='jmeno',
                            how='left')

In [98]:
vystup_left_join_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25,Brno,100.0
1,Bob,30,Praha,NaN
2,Charlie,35,Plzen,NaN
3,David,40,Ostrava,200.0


V ukázce výš je použitý *left join*.

Tedy ve výsledku uvidíš **celou první** (levou tabulku) a z druhé pouze ty záznamy, které mají ve spojovacím sloupci `jmeno` společnou hodnotu.

In [99]:
vystup_left_join_obr_df = merge(uzivatele_tab_2_df,
                            uzivatele_tab_1_df,
                            on='jmeno',
                            how='left')

In [101]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


In [100]:
vystup_left_join_obr_df

,jmeno,pocet_prijemcu,vek,mesto
0,Alice,100,25.0,Brno
1,David,200,40.0,Ostrava
2,Emma,150,NaN,NaN
3,Frank,250,NaN,NaN


### Spojení pomocí indexů, `JOIN`

---

Tato metoda slouží k propojení dvou DataFrame objektů na základě **jejich indexů nebo hodnot**.

Je velice podobná funkci `merge` ale je přímo součástí `DataFrame` objektu a je snazší ji aplikovat:

In [102]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
     'vek': [25, 30, 35]
}

In [103]:
uzivatele_2 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
    'pocet_prijemcu': [100, 150, 200]
}

In [117]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [105]:
df_uzivatele_1

,jmeno,vek
0,Alice,25
1,Bob,30
2,Petr,35


In [106]:
df_uzivatele_2

,jmeno,pocet_prijemcu
0,Alice,100
1,Bob,150
2,Petr,200


In [107]:
df_uzivatele_2.index.names

FrozenList([None])

In [108]:
df_uzivatele_2.index

RangeIndex(start=0, stop=3, step=1)

In [109]:
uzivatele_1_df = df_uzivatele_1.set_index('jmeno')

In [110]:
uzivatele_2_df = df_uzivatele_2.set_index('jmeno')

In [111]:
uzivatele_1_df

,vek
jmeno,
Alice,25
Bob,30
Petr,35


In [112]:
uzivatele_2_df

,pocet_prijemcu
jmeno,
Alice,100
Bob,150
Petr,200


In [113]:
vysledek_join_metody = uzivatele_1_df.join(uzivatele_2_df)

In [114]:
vysledek_join_metody

,vek,pocet_prijemcu
jmeno,,
Alice,25,100
Bob,30,150
Petr,35,200


Hlavní rozdíl mezi těmito funkcemi je způsob určení **sloupce nebo sloupců**, podle kterých se má propojení provést.

1. Metoda `join()` propojuje DataFrame objekty **na základě jejich Indexů**.
2. Funkce `merge()` umožňuje propojit `DataFrame` objekty **na základě hodnoty v jednom nebo více sloupcích**.

### Souhrn ke spojování

---

| Objekt | Kdy jej použít? | Směr spojení |
| :- | :- | :- |
| `concat` | Když chceš poskládat tabulky na sebe (řádky) nebo vedle sebe (sloupce). | Vertikálně / Horizontálně | 
| `merge` |	Když chcete spojit tabulky podle společného klíče/sloupce, podobně jako SQL JOIN. | Horizontálně |
| `join` | Když chcete spojit tabulky podle společného Indexu (ne klíče/sloupce). | Horizontálně |
| `append` | (Zastaralé) Používalo se k přidání řádků, nyní používejte concat. | Vertikálně |

<br>

**🧠 CVIČENÍ 🧠, procvič si spojování**

Tvým úkolem je propojit tyto tabulky podle zadání:
1. Použij spojovací funkci pro spojení objektů typu `DataFrame` , které vytvoříš z proměnných `data_zamestnanci_1` a `data_zamestnanci_2`,
2. použij spojovací funkci, které zadáš vzniklý objekt typu `DataFrame`  z bodu 1. a druhý `DataFrame`, který vytvoříš z proměnné `data_oddeleni` (spoj přes sloupeček `"oddělení_id"`),
3. vytvoř výsledný objekt typu `DataFrame` , který obsahuje pouze zaměstnance z oddělení `IT`.

In [118]:
data_zamestnanci_1 = {
    'id': [1, 2, 3],
    'jméno': ['Jan', 'Marie', 'Petr'],
    'příjmení': ['Novák', 'Svobodová', 'Dvořák'],
    'oddělení_id': [100, 200, 100]
}

data_zamestnanci_2 = {
    'id': [4, 5],
    'jméno': ['Anna', 'Josef'],
    'příjmení': ['Kopecká', 'Vondráček'],
    'oddělení_id': [200, 100]
}

data_oddeleni = {
    'oddělení_id': [100, 200],
    'název_oddělení': ['IT', 'HR']
}

In [125]:
zamestnanci_1_df = DataFrame(data_zamestnanci_1)
zamestnanci_2_df = DataFrame(data_zamestnanci_2)

oddeleni_df = DataFrame(data_oddeleni)
spojeni_zamestnanci_df = concat((zamestnanci_1_df, zamestnanci_2_df), axis=0)

zamestnanci_oddeleni_df = merge(oddeleni_df, spojeni_zamestnanci_df, on='oddělení_id')

In [128]:
# merge?

In [132]:
def jen_it(kompletni_df,
           hledany_sloupec: str,
           hledana_hodnota: str) -> DataFrame:
    return kompletni_df[kompletni_df[hledany_sloupec] == hledana_hodnota]  # isin

# zamestnanci_oddeleni_df[zamestnanci_oddeleni_df['název_oddělení'] == 'IT']

In [133]:
jen_it(kompletni_df=zamestnanci_oddeleni_df,
       hledany_sloupec='název_oddělení',
       hledana_hodnota='IT')

,oddělení_id,název_oddělení,id,jméno,příjmení
0,100,IT,1,Jan,Novák
1,100,IT,3,Petr,Dvořák
2,100,IT,5,Josef,Vondráček


<details>
    <summary>▶️ Řešení</summary>
    
```python
df_zamestnanci_1 = pd.DataFrame(data_zamestnanci_1)
df_zamestnanci_2 = pd.DataFrame(data_zamestnanci_2)
df_oddeleni = pd.DataFrame(data_oddeleni)

df_vsichni_zamestnanci = pd.concat([df_zamestnanci_1, df_zamestnanci_2], ignore_index=True)

# df1 = pd.concat(
#     [pd.DataFrame(data) for data in [data_zamestnanci_1, data_zamestnanci_2]]
# ).reset_index(drop = True).set_index('id')

df_vsechna_oddeleni = pd.merge(df_vsichni_zamestnanci, df_oddeleni, on="oddělení_id")

df_jen_it = df_vsechna_oddeleni[df_vsechna_oddeleni["název_oddělení"] == "IT"]
```
</details>

## Agregace

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.HtBtNx6dodlNptXV1QGHWQHaG0%26pid%3DApi&f=1&ipt=3a4d1dfa14a495127b4ece5cf689f00ed0c9e0836853358412e18161ae7c505f&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>


*Seskupování* neboli **agregace** jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

---

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [134]:
from pandas import read_csv

In [137]:
nemovitosti_df = read_csv("../onsite/housing.csv")

In [138]:
nemovitosti_df.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade
0,1,221900.0,3,1.00,1180,5650,1,3,7
1,2,538000.0,3,2.25,2570,7242,2,3,7
2,3,180000.0,2,1.00,770,10000,1,3,6
3,4,604000.0,4,3.00,1960,5000,1,5,7
4,5,510000.0,3,2.00,1680,8080,1,3,8


In [139]:
nemovitosti_df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade
count,10.00000,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,5.50000,4.380750e+05,3.100000,2.100000,2002.500000,16846.200000,1.300000,3.200000,7.400000
std,3.02765,3.137603e+05,0.567646,1.094177,1305.367658,29937.105556,0.483046,0.632456,1.349897
min,1.00000,1.800000e+05,2.000000,1.000000,770.000000,5000.000000,1.000000,3.000000,6.000000
25%,3.25000,2.365000e+05,3.000000,1.125000,1305.000000,6624.750000,1.000000,3.000000,7.000000
50%,5.50000,3.074250e+05,3.000000,2.125000,1747.500000,7356.000000,1.000000,3.000000,7.000000
75%,7.75000,5.310000e+05,3.000000,2.437500,1942.500000,9303.250000,1.750000,3.000000,7.000000
max,10.00000,1.225000e+06,4.000000,4.500000,5420.000000,101930.000000,2.000000,5.000000,11.000000


In [141]:
nemovitosti_df.loc[:, "price"].max()

1225000.0

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

### Seskupování GROUPBY

---

Metoda `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [142]:
from pandas import DataFrame

In [143]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
     'číselná hodnota': range(6)},
)

In [144]:
pokus_s_cisly_df

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


#### Klasické seskupení podle sloupečku

---

In [146]:
# pokus_s_cisly_df.groupby?

In [147]:
pokus_s_cisly_df.groupby("klíč")

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [149]:
type(pokus_s_cisly_df.groupby("klíč"))

pandas.core.groupby.generic.DataFrameGroupBy

Metoda standardně vrací `DataFrameGroupBy`.

<br>

In [150]:
pokus_s_cisly_df.groupby("klíč").sum()

,číselná hodnota
klíč,
A,3
B,5
C,7


Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

<br>

#### Sloupečkové označování

---

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [151]:
pokus_s_cisly_df

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [152]:
pokus_s_cisly_df.groupby("klíč")

In [153]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"]

In [155]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"].sum()

klíč
A    3
B    5
C    7
Name: číselná hodnota, dtype: int64

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `sum`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [157]:
for (klic, hodnota) in pokus_s_cisly_df.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota}")

Klic: A; Hodnota=  klíč  číselná hodnota
0    A                0
3    A                3
Klic: B; Hodnota=  klíč  číselná hodnota
1    B                1
4    B                4
Klic: C; Hodnota=  klíč  číselná hodnota
2    C                2
5    C                5


<br>

### Agregace s metodami

---

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

#### Metoda AGGREGATE

---

In [158]:
import numpy

In [159]:
rng = numpy.random.RandomState(0)

In [160]:
from pandas import DataFrame

In [164]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C', 'C'],
     'data_1': range(7),
     'data_2': rng.randint(0, 10, 7)}
)

In [165]:
pokus_s_cisly_df

,klíč,data_1,data_2
0,A,0,8
1,B,1,8
2,C,2,1
3,A,3,6
4,B,4,7
5,C,5,7
6,C,6,8


Statistické metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [166]:
pokus_s_cisly_df.groupby("klíč").aggregate("sum")

,data_1,data_2
klíč,,
A,3,14
B,5,15
C,13,16


In [167]:
pokus_s_cisly_df.groupby("klíč").aggregate(numpy.median)

,data_1,data_2
klíč,,
A,1.5,7.0
B,2.5,7.5
C,5.0,7.0


In [168]:
pokus_s_cisly_df.groupby("klíč").aggregate(["min", numpy.median, max])

data_1            data_2           
        min median max    min median max
klíč                                    
A         0    1.5   3      6    7.0   8
B         1    2.5   4      7    7.5   8
C         2    5.0   6      1    7.0   8

In [ ]:
pokus_s_cisly_df

Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

| pobocka | ovoce | prodane_mnozstvi | cena |
| :- | :- | :- | :- |
| A | apple | 10 | 20 |

In [170]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [171]:
ovoce_df = DataFrame(ovoce_data)

In [172]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [176]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': "sum"})  # .sum()

,prodane_mnozstvi
pobocka,
A,25
B,38
C,45


In [177]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': sum, 'cena': 'mean'})

,prodane_mnozstvi,cena
pobocka,,
A,25,16.0
B,38,16.0
C,45,16.5


1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Metoda FILTER

---

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání **seskupených dat** podle splnění **určité podmínky**.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [178]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [179]:
ovoce_df = DataFrame(ovoce_data)

In [180]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [181]:
def vyber_s_prodejem_nad_limit(data: DataFrame,
                               limit: int = 30) -> bool:
    return data['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_s_prodejem_nad_limit`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [182]:
ovoce_df.groupby('pobocka').filter(vyber_s_prodejem_nad_limit)

,pobocka,ovoce,prodane_mnozstvi,cena
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci (`vyber_pouze_pobocky_s_prodejem_nad_limit`),
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

### Metoda TRANSFORM

---

Zatímco předchozí výsledky *agregace* pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [183]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [184]:
def vrat_procenta_z_celkoveho_prodeje(udaj: Series) -> float:
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `vrat_procenta_z_celkoveho_prodeje`:

In [185]:
ovoce_df['Procento z celkoveho prodeje'] = ovoce_df.groupby('pobocka')['prodane_mnozstvi'] \
    .transform(vrat_procenta_z_celkoveho_prodeje)

In [186]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena,Procento z celkoveho prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `procento_z_celkoveho_prodeje`.

### Metoda `APPLY`

---

Metoda `apply` ti také dovolí, používat uživatelem definované funkce na seskupená data.

Následně vrací objekt knihovny `pandas` (buď `DataFrame`, nebo `Series`, a nebo skalární hodnota).

In [187]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena,Procento z celkoveho prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


In [190]:
def vypocitej_vydelek_za_artikl(seskupene: DataFrame):
    seskupene['vydelek_za_artikl'] = seskupene['prodane_mnozstvi'] * seskupene['cena']
    return seskupene

<br>

Metoda `apply` je **obecnější a flexibilnější** než `transform`.

`apply` umožňuje použít uživatelem definovanou funkci na každou skupinu po seskupení dat pomocí `groupby`.

Výsledek metody `apply` může mít jiný tvar než původní data.

In [195]:
ovoce_df.groupby('pobocka', group_keys=False).apply(vypocitej_vydelek_za_artikl)

,pobocka,ovoce,prodane_mnozstvi,cena,Procento z celkoveho prodeje,vydelek_za_artikl
0,A,jablko,10,20,40.0,200
1,A,banan,15,12,60.0,180
2,B,jablko,8,22,21.1,176
3,B,banan,30,10,78.9,300
4,C,jablko,20,18,44.4,360
5,C,banan,25,15,55.6,375


In [197]:
# ovoce_df.groupby?

Od posledních verzí frameworku (`1.5.0` a vyšší) platí, že pokud bude výsledkem `DataFrame` nebo `Series` musíš uvést argument pro `group_keys=True`).

<br>

In [ ]:
vystup = df_ovoce.groupby('pobocka', group_keys=True).apply(vypocitej_vydelek_za_artikl)

In [ ]:
vystup

Na první pohled vypadají metody `apply` a `transform` docela podobně.

Metoda `transform` je trochu omezenější než `apply`.

Slouží k aplikaci uživatelem definované nebo vestavěné funkce **na každý prvek** skupiny po seskupení s `groupby`.

`transform` musí vracet hodnotu stejného tvaru jako vstupní data.

Výsledek metody `transform` **má stejný tvar jako původní data**.

#### Apply
* potřebuješ výsledek, který **má jiný tvar než původní data**,
* umí zpracovat **více sloupečků současně**.

#### Transform
* potřebuješ výsledek, který **má stejný tvar jako původní data**,
* umí zpracovat **pouze jeden sloupeček**.

In [ ]:
df_rozdily = DataFrame({
    'KLIC': ['A','B','C'] * 3,
    'A': numpy.arange(9),
    'B': [1,2,3] * 3,
})

In [ ]:
df_rozdily

#### `transform` vrací výsledky ve stejném tvaru

---

In [ ]:
def vypocitej_sumu(data):
    return data.sum()

In [ ]:
seskup_df_rozdily_apply = df_rozdily.groupby('KLIC')['A'].apply(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_apply

In [ ]:
seskup_df_rozdily_trans = df_rozdily.groupby('KLIC')['A'].transform(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_trans

#### `apply` umí pracovat s více sloupečky, `transform` jen s jedním

---

In [ ]:
df_rozdily

In [ ]:
def vypocitej_rozdil(data):
    return data['B'] - data['A']

In [ ]:
df_rozdily.groupby('KLIC').apply(vypocitej_rozdil)

In [ ]:
# df_rozdily.groupby('KLIC').transform(vypocitej_rozdil)

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí GroupBy a agregační funkce**

Máš zadaný takový datový set.

Následně:
1. Pomocí metody `filter` vyber prodejny, které prodaly **alespoň 30 produktů**,
2. na filtrovaném datasetu použijte metodu `apply` pro výpočet **celkového příjmu z prodeje pro každý obchod**.

In [198]:
from pandas import DataFrame

df_prodej_hardware = DataFrame({
    'prodejna_id': (5, 4, 1, 5, 5, 1, 4, 2, 5, 1, 3, 1, 3, 4, 2, 1, 5, 4, 1, 5),
    'transakce_id': (1278, 1216, 1866, 1872, 1797, 1272, 1880, 1061, 1595, 1879, 1728,
       1341, 1396, 1698, 1018, 1176, 1611, 1395, 1444, 1232),
    'predmet_prodeje': ('grafická_karta', 'SSD', 'RAM', 'procesor', 'grafická_karta',
       'základní_deska', 'SSD', 'SSD', 'grafická_karta', 'RAM',
       'grafická_karta', 'procesor', 'grafická_karta', 'SSD',
       'grafická_karta', 'RAM', 'základní_deska', 'HDD', 'grafická_karta',
       'RAM'),
    'pocet_prodanych_ks': (1,  5,  6,  6,  3,  7,  9, 10,  8,  6,  8,  5,  8, 10,  4, 10,  8,
       10,  2,  5),
    'cena_predmetu': (19500.69874949, 19731.10951735, 14114.15342339, 10953.87914371,
        6535.78851758, 16369.00288429, 13852.2578648 ,  3671.03031723,
       18263.08009763, 16539.476237  , 19021.09830919, 14651.53041357,
       12461.59632075,  8655.73920767, 18688.2054254 , 17388.24584526,
        1381.76406707,  1014.1560027 ,  7841.03565412, 16305.78995025)
})

In [210]:
def vyber_prodejny_s_vice_nez_triceti_produkty(
    data: DataFrame, limit: int = 30
):
    return data['pocet_prodanych_ks'].sum() >= limit

In [211]:
hledane_prodejny = df_prodej_hardware.groupby('prodejna_id').filter(
    vyber_prodejny_s_vice_nez_triceti_produkty
)

In [212]:
hledane_prodejny

,prodejna_id,transakce_id,predmet_prodeje,pocet_prodanych_ks,cena_predmetu
0,5,1278,grafická_karta,1,19500.698749
1,4,1216,SSD,5,19731.109517
2,1,1866,RAM,6,14114.153423
3,5,1872,procesor,6,10953.879144
4,5,1797,grafická_karta,3,6535.788518
5,1,1272,základní_deska,7,16369.002884
6,4,1880,SSD,9,13852.257865
8,5,1595,grafická_karta,8,18263.080098
9,1,1879,RAM,6,16539.476237
11,1,1341,procesor,5,14651.530414


In [209]:
df_prodej_hardware.groupby('prodejna_id').sum()

,transakce_id,predmet_prodeje,pocet_prodanych_ks,cena_predmetu
prodejna_id,,,,
1,8978,RAMzákladní_deskaRAMprocesorRAMgrafická_karta,36,86903.444458
2,2079,SSDgrafická_karta,14,22359.235743
3,3124,grafická_kartagrafická_karta,16,31482.694630
4,6189,SSDSSDSSDHDD,34,43253.262593
5,9385,grafická_kartaprocesorgrafická_kartagrafická_k...,31,72941.000526


In [257]:
agregovano = df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()
odfiltrovano = agregovano[agregovano > 30]

print(odfiltrovano)

prodejna_id
1    36
4    34
5    31
Name: pocet_prodanych_ks, dtype: int64


In [200]:
df_prodej_hardware.head()

,prodejna_id,transakce_id,predmet_prodeje,pocet_prodanych_ks,cena_predmetu
0,5,1278,grafická_karta,1,19500.698749
1,4,1216,SSD,5,19731.109517
2,1,1866,RAM,6,14114.153423
3,5,1872,procesor,6,10953.879144
4,5,1797,grafická_karta,3,6535.788518


<details>
    <summary>▶️ Řešení</summary>
    
```python
def vyber_prodej_vetsi_nez_limit(data, limit: int = 30):
    return data['pocet_prodanych_ks'].sum() > limit
    
def vypocitej_celkovy_vydelek_prodejny(skupina):
    return (skupina['pocet_prodanych_ks'] * skupina['cena_predmetu']).sum()

df_filtr_hardware = df_prodej_hardware.groupby('prodejna_id') \
                        .filter(vyber_prodej_vetsi_nez_limit)

#         cilove_prodejny = df_prodej_hardware.groupby('prodejna_id') \
# .filter(lambda prodejna: prodejna.pocet_prodanych_ks.sum() >= 30)


celkovy_vydelek = df_filtr_hardware.groupby('prodejna_id') \
                        .apply(vypocitej_celkovy_vydelek_prodejny)

# prodeje_prodejen = cilove_prodejny.groupby('prodejna_id') \
# .aggregate({'pocet_prodanych_ks': sum})

# df_prodej_hardware["prodejna_id"].unique()

# prijmy_prodejen = pd.DataFrame(df_prodej_hardware.groupby('prodejna_id')['cena_predmetu'] \
#                 .apply(lambda prodejna: prodejna.sum())) \
#                 .rename(columns = { 'cena_predmetu': 'prijem_z_prodeje' } ) \
#                 .round( 2 )
```
</details>


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.P16hnt14CM-mJ5eAKykxLAHaHa%26pid%3DApi&f=1&ipt=27e3f5b23fe44567ad0730192d5dcb13c7c23aa6e87579d16df05e27461dc12c&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

## Pivot tabulky

---


*Pivot tabulky* (*kontingenční tabulky*) jsou užitečné pro přehledné zobrazení a analýzu dat z tabulkových zdrojů.

Pomocí pivot tabulek můžeš **seskupit data podle určitých kategorií** a provést agregaci hodnot.

Nejprve si představ situaci bez pivot tabulek, pomocí ukázky níže:

In [ ]:
!pip install seaborn

In [214]:
import seaborn

<br>

V této ukazce použiješ vzorová data týkající se [nehody lodi Titanic](https://en.wikipedia.org/wiki/Sinking_of_the_Titanic):

In [215]:
df_titanic = seaborn.load_dataset('titanic')

In [217]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


<br>

Pro jakoukoliv pokročilou analýzu dat, potřebuješ údaj seskupit.

Tak jak jsi do prováděl doposud, pomocí `groupby`:

In [219]:
df_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [220]:
df_titanic["sex"].unique()

array(['male', 'female'], dtype=object)

In [221]:
df_titanic.groupby('sex')

<br>

Potřebuješ průměrnou hodnotu, ať je na sloupečku `survived` vidět poměr přeživších:

In [222]:
df_titanic.groupby('sex')['survived'].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

<br>

Pro lepší čitelnost **zaokrouhlím výsledek na dvě desetinné čísla**:

In [223]:
df_titanic.groupby('sex')['survived'].mean().round(2)

sex
female    0.74
male      0.19
Name: survived, dtype: float64

Takový průzkum z této studie ti dá jasný pohled na věc:
* 3 ze 4 žen přežily,
* 1 z 5 mužů přežil.

<br>

Pokud budeš potřebovat detailnější analýzy, budeš potřebovat více dat.

Třeba situaci, kde kromě pohlaví, bereš v potaz **třídu cestujících** `class`:

In [224]:
df_titanic["class"].unique()

['Third', 'First', 'Second']
Categories (3, object): ['First', 'Second', 'Third']

In [225]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean()

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

In [ ]:
type(df_titanic.groupby(['sex', 'class'])[['survived']])

In [ ]:
type(df_titanic.groupby(['sex', 'class'])['survived'].mean())

In [226]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean().round(2)

survived
sex    class           
female First       0.97
       Second      0.92
       Third       0.50
male   First       0.37
       Second      0.16
       Third       0.14

<br>

Metodou `unstack` si můžeš vytvořit nové sloupečky, které jsou postavené na novém Indexu, nebo Indexech (*Multiindex*):

In [227]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


Takový průzkum ti dá skutečně lepší pohled na věc.

Současně ale roste **náročnost ohlášení**.

Zápis "bobtná" a **stává se náročnějším na přečtení a pochopení**.

<br>

### Pivot tabulka

---

Podobné řešení ti nabízí funkce `pivot_table`:

In [228]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


In [229]:
df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


In [230]:
df_titanic.index

RangeIndex(start=0, stop=891, step=1)

Zásádním rozdílem je ovšem **čitelnost**, kterou máš pro tuto variantu zápisu.

```
Clarity beats purity
```

Pomocí vhodných argumentů, můžeš doplnit vysvětlivky tam, kde funkce `groupby` nemohla.

Stejně platí, že pokud budeš potřebovat **další Index**, můžeš si pomoci funkcí `cut`:

In [231]:
df_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [232]:
df_titanic['age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64

In [233]:
from pandas import cut

In [234]:
age = cut(df_titanic['age'], [1, 40, 80])

In [235]:
age  # < xxx , xxx )

0      (1.0, 40.0]
1      (1.0, 40.0]
2      (1.0, 40.0]
3      (1.0, 40.0]
4      (1.0, 40.0]
          ...     
886    (1.0, 40.0]
887    (1.0, 40.0]
888            NaN
889    (1.0, 40.0]
890    (1.0, 40.0]
Name: age, Length: 891, dtype: category
Categories (2, interval[int64, right]): [(1, 40] < (40, 80]]

In [237]:
vystup_df = df_titanic.pivot_table('survived', index=['sex', age], columns='class').round(2)

In [238]:
vystup_df

class            First  Second  Third
sex    age                           
female (1, 40]    0.97    0.93   0.47
       (40, 80]   0.96    0.85   0.11
male   (1, 40]    0.50    0.11   0.16
       (40, 80]   0.28    0.10   0.06

In [239]:
vystup_df.index

MultiIndex([('female',  (1, 40]),
            ('female', (40, 80]),
            (  'male',  (1, 40]),
            (  'male', (40, 80])],
           names=['sex', 'age'])

Pomocí funkce `cut` doplníš tabulku na **MultiIndex**.

Ten nyní vytvoří i rozsah pro věkovou kategorii.

<br>

**🧠 CVIČENÍ 🧠, procvič si pivot tabulky**

Ze zadaného datasetu vytvoř **pivot tabulku**, která zobrazí **počet prodaných kusů ovoce** pro **každý obchod** a **druh ovoce**.

In [240]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
# pivot_table()
#              prodane_mnozstvi      jablko banan
# Pobocka
#    A 
#    B
#    C 

In [241]:
ovoce_df = DataFrame(ovoce_data)

In [243]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [244]:
pt = ovoce_df.pivot_table(values='prodane_mnozstvi',
                     index='pobocka',
                     columns='ovoce')

In [250]:
pt.index.names = ['pobocka_id']

In [251]:
pt

ovoce,banan,jablko
pobocka_id,,
A,15,10
B,30,8
C,25,20


<details>
    <summary>▶️ Řešení</summary>
    
```python
pivot_tabulka = df_ovoce.pivot_table(values="prodane_mnozstvi", index="pobocka", columns="ovoce")
```
</details>

---